In [9]:
require('tidyverse')

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.1     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


In [4]:
# find REPEAT visits
require('data.table')

phase_meno <- fread("../data/phase_meno_qc.txt", header=TRUE, data.table=FALSE)
hormone_drug <- fread("../data/id_to_hormone_drug.txt", header=TRUE, data.table=FALSE)

Loading required package: data.table


In [10]:
repeat_visit <- unique(filter(phase_meno, (! is.na(visit)) &( visit %in% c(1,2)))$IID)
length(repeat_visit)

[1] 19104

In [18]:
# 19104 individuals have multiple visits
repeat_ppl <- filter(phase_meno, IID %in% repeat_visit) %>% unique()

In [22]:
dim(repeat_ppl)
repeat_ppl2 <- filter(repeat_ppl, preg == 0 & qc_fail==0)
dim(repeat_ppl2)
table(repeat_ppl2$meno.label, repeat_ppl2$visit)

[1] 54998    23

[1] 52514    23

               
                   0    1    2
  likely_meno   1024   57   19
  male            22   17    9
  missing       2076  852 1420
  missing_age      3    2    0
  peri             0    0  256
  post          9120 6973 8584
  pre           8024 2256 1716
  premature      834  680  858
  surgical_meno 3286 2187 2239

In [27]:
pre_meno <- filter(repeat_ppl2, meno.label=="pre")
v0 <- unique(filter(pre_meno, visit==0)$IID)
v1 <- unique(filter(pre_meno, visit==1)$IID)
v2 <- unique(filter(pre_meno, visit==2)$IID)

v1_v2_i <- intersect(intersect(v1, v2), v0)
length(v1_v2_i)
v1_v2_u <- union(v1, v2)
length(v1_v2_u)
mult_v <- intersect(v0, v1_v2_u)
length(mult_v)

# cycle info
# normalized day in cycle

[1] 165

[1] 1626

[1] 1573

In [35]:
mult_v_dat <- filter(repeat_ppl2, IID %in% mult_v)

# look at inter-individual variability
menstrual_dat <- filter(mult_v_dat, !is.na(menstrual_phase))

dim(menstrual_dat) # 5311 
table(menstrual_dat$pill)
table(menstrual_dat$hrt)

[1] 5311   23


  -9    0    1 
 184 4147  287 


-9  0  1 
 4 70 66 

In [39]:

biomarker_phe <- fread("../phe_extraction/sm_biomarker_tab.txt", data.table=FALSE)
head(biomarker_phe)
filt_biom <- filter(biomarker_phe, f.eid %in% menstrual_dat$IID)

f.eid,30600,30610,30620,30630,30640,30650,30660,30670,30680,⋯,30800,30810,30820,30830,30840,30850,30860,30870,30880,30890
5586147,NA,114.3,100.78,NA,0.807,57.4,2.24,6.19,NA,⋯,NA,NA,NA,NA,8.88,11.419,NA,2.509,350.3,67.8
5144865,NA,59.6,20.66,NA,0.850,15.6,1.40,4.38,NA,⋯,NA,NA,NA,NA,7.80,11.395,NA,4.416,414.2,26.2
2845962,46.07,83.4,43.99,1.300,1.180,33.1,2.09,6.61,2.395,⋯,NA,1.045,12.7,36.27,12.13,12.911,75.09,1.792,422.2,56.6
3390045,47.45,41.4,35.54,1.840,1.335,28.0,NA,4.22,2.342,⋯,NA,1.068,NA,24.22,4.43,8.204,72.45,1.460,524.3,77.8
4264965,45.75,61.1,18.10,1.985,1.022,23.8,1.59,3.90,2.505,⋯,NA,1.201,NA,89.68,8.26,0.675,78.93,1.200,191.2,12.1
3131746,49.71,108.5,21.24,1.679,1.057,NA,NA,5.51,2.576,⋯,NA,1.430,NA,38.54,NA,0.986,82.51,2.215,368.9,58.9


In [43]:
# NEED THE BIOMARKER TAB WITH THE FULL DATASET
biomarker <- read.table("../phe_extraction/list_biomarker.txt", header=TRUE)
rownames(biomarker) <- biomarker$Field
combined_mat2 <- filt_biom
colnames(combined_mat2) <- c("IID", sapply(biomarker[colnames(combined_mat2)[2:ncol(combined_mat2)],]$name, as.character))
head(combined_mat2)


IID,Albumin,Alkaline phosphatase,Alanine aminotransferase,Apolipoprotein A,Apolipoprotein B,Aspartate aminotransferase,Direct bilirubin,Urea,Calcium,⋯,Oestradiol,Phosphate,Rheumatoid factor,SHBG,Total bilirubin,Testosterone,Total protein,Triglycerides,Urate,Vitamin D
4264965,45.75,61.1,18.10,1.985,1.022,23.8,1.59,3.90,2.505,⋯,NA,1.201,NA,89.68,8.26,0.675,78.93,1.200,191.2,12.1
2331876,40.44,49.4,13.32,1.811,0.949,22.4,1.85,3.98,2.221,⋯,447.5,1.033,NA,141.93,9.66,NA,69.20,0.969,247.6,89.9
1289179,44.38,75.0,9.95,1.364,0.672,15.9,1.27,3.18,2.350,⋯,NA,1.203,NA,73.49,6.01,0.787,70.27,1.214,277.1,28.0
1964158,42.76,66.3,12.43,1.680,0.929,16.8,NA,3.67,2.433,⋯,931.1,1.296,NA,41.77,4.03,NA,68.45,0.915,194.3,64.7
2124623,43.51,61.2,15.51,1.340,0.995,17.0,2.23,3.89,2.315,⋯,326.2,1.076,NA,46.87,11.09,0.805,78.90,0.627,310.8,14.9
1248443,44.13,91.8,13.75,1.399,0.819,15.2,1.57,2.44,2.271,⋯,589.2,0.975,NA,95.59,7.76,1.554,70.30,0.660,203.1,68.1


f.eid,30600,30610,30620,30630,30640,30650,30660,30670,30680,⋯,30800,30810,30820,30830,30840,30850,30860,30870,30880,30890
4264965,45.75,61.1,18.10,1.985,1.022,23.8,1.59,3.90,2.505,⋯,NA,1.201,NA,89.68,8.26,0.675,78.93,1.200,191.2,12.1
2331876,40.44,49.4,13.32,1.811,0.949,22.4,1.85,3.98,2.221,⋯,447.5,1.033,NA,141.93,9.66,NA,69.20,0.969,247.6,89.9
1289179,44.38,75.0,9.95,1.364,0.672,15.9,1.27,3.18,2.350,⋯,NA,1.203,NA,73.49,6.01,0.787,70.27,1.214,277.1,28.0
1964158,42.76,66.3,12.43,1.680,0.929,16.8,NA,3.67,2.433,⋯,931.1,1.296,NA,41.77,4.03,NA,68.45,0.915,194.3,64.7
2124623,43.51,61.2,15.51,1.340,0.995,17.0,2.23,3.89,2.315,⋯,326.2,1.076,NA,46.87,11.09,0.805,78.90,0.627,310.8,14.9
1248443,44.13,91.8,13.75,1.399,0.819,15.2,1.57,2.44,2.271,⋯,589.2,0.975,NA,95.59,7.76,1.554,70.30,0.660,203.1,68.1


In [49]:
comb_dat <- inner_join(select(menstrual_dat, IID, visit, menstrual_phase), combined_mat2)

comb_dat2 <- filter(comb_dat, !is.na("Albumin "))
head(comb_dat2)
dim(comb_dat2)

Joining, by = "IID"


IID,visit,menstrual_phase,Albumin,Alkaline phosphatase,Alanine aminotransferase,Apolipoprotein A,Apolipoprotein B,Aspartate aminotransferase,Direct bilirubin,⋯,Oestradiol,Phosphate,Rheumatoid factor,SHBG,Total bilirubin,Testosterone,Total protein,Triglycerides,Urate,Vitamin D
5733299,0,luteal,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5733299,0,follicular,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5733299,1,luteal,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5733299,1,follicular,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2070311,0,follicular,39.04,65.9,15.15,1.21,1.047,21.2,1.09,⋯,552.5,1.213,NA,35.24,5.62,NA,66.95,1.372,269.4,52.4
2070311,1,follicular,39.04,65.9,15.15,1.21,1.047,21.2,1.09,⋯,552.5,1.213,NA,35.24,5.62,NA,66.95,1.372,269.4,52.4


[1] 5311   33

In [ ]:
com